<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-24 09:49:58
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.40 C
-------------------
Today PnL: -99.65 K (-0.7%)
Current PnL: -33.99 L (-21.24%)
CY Booked + Current PnL: -18.31 L (-11.44%)
-------------------
Total profit:  1.36 L
Total loss:  -35.34 L
-------------------
Total Booked + Current PnL: 8.67 L (6.52%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.16%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.01 C (71.63%)
Deployed:  1.33 C
Current:  1.40 C
CAGR/XIRR %: 2.8%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-1.61,2.76,18.31,21.57,18435.0,2700.0,100680.0,397.04,-11.47,56.0,H-SC,6.16,167.0,0.15,0.73,46.92,XR,ATH,FINANCE
18,COALINDIA,0.55,5.34,11.71,17.67,18559.0,8030.0,158490.0,478.51,26.26,57.0,L-LC,11.56,185.0,0.43,1.15,27.23,XY25,ATH,MINING
77,TTKPRESTIG,0.01,-31.13,45.32,0.09,31457.0,-31367.0,69410.0,770.00,61.15,29.0,M-SC,10.52,250.0,-1.00,0.50,0.01,OX40N,NTT,DURABLES
36,ICICIGI,-1.15,2.31,16.65,19.34,34087.0,4619.0,204728.0,2252.93,-13.95,56.0,X-MC,1.68,66.0,0.14,1.48,18.61,X40,ATH,INSURANCE
51,MEDANTA,-1.90,-4.50,29.92,24.08,36276.0,-5707.0,121243.0,1486.00,-5.20,51.0,X-SC,9.37,87.0,-0.16,0.88,11.05,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANDHANBNK,3.02,-13.94,123.59,92.42,296277.0,-38833.0,239726.0,400.0,105.46,75.0,H-SC,6.52,174.0,-0.13,1.74,33.40,XY24,NTT,BANKS
62,SATIN,1.12,-13.58,75.70,51.83,180741.0,-37525.0,238759.0,274.0,-22.47,54.0,H-SC,3.99,159.0,-0.21,1.73,16.31,XY24,NTT,FINANCE
82,VOLTAS,0.77,21.26,19.74,45.20,45896.0,40758.0,232500.0,1856.0,38.75,65.0,X-MC,7.41,79.0,0.89,1.68,29.50,XY25,NTT,AC
19,COFFEEDAY,0.63,-46.16,151.26,35.27,92470.0,-52416.0,61133.0,80.0,-57.75,47.0,L-SC,14.27,266.0,-0.57,0.44,49.62,XR,NTT,HOTELS
65,SIEMENS,0.60,-10.74,40.62,25.51,67470.0,-19995.0,166100.0,4671.5,39.40,63.0,H-LC,2.46,51.0,-0.30,1.20,22.40,AR,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,-4.14,-11.28,39.58,23.83,84953.0,-27289.0,214635.0,1908.19,-6.95,19.0,X-LC,3.81,8.0,-0.32,1.56,2.98,X40,ATH,IT
25,EASEMYTRIP,-3.50,-51.32,199.32,45.70,171547.0,-90749.0,86066.0,26.40,-10.53,65.0,M-SC,18.82,204.0,-0.53,0.62,41.80,XY24,NTT,TRAVEL
71,TANLA,-3.50,-49.57,335.66,119.69,458300.0,-134224.0,136537.0,1943.92,-49.08,36.0,H-SC,19.07,147.0,-0.29,0.99,7.86,AR,ATH,IT
42,INFY,-3.36,-12.67,53.71,34.24,146767.0,-39630.0,273258.0,1972.00,-31.42,18.0,X-LC,6.31,3.0,-0.27,1.98,0.00,X40,NTT,IT
73,TCS,-3.18,-25.73,66.40,23.58,213341.0,-111333.0,321296.0,4311.59,-35.76,23.0,X-LC,6.96,2.0,-0.52,2.33,0.00,X40,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-1.11,3.35,101.17,107.91,154377.0,4948.0,152592.0,1641.55,-7.15,59.0,M-SC,7.92,220.0,0.03,1.11,14.80,OX40N,ATH,CABLES
77,TTKPRESTIG,0.01,-31.13,45.32,0.09,31457.0,-31367.0,69410.0,770.00,61.15,29.0,M-SC,10.52,250.0,-1.00,0.50,0.01,OX40N,NTT,DURABLES
66,SIS,-0.33,-31.39,76.74,21.26,58461.0,-34851.0,76181.0,528.00,1787.56,32.0,H-SC,4.43,168.0,-0.60,0.55,5.02,OX40N,NTT,MISC
40,INDIGOPNTS,-0.57,-32.84,48.92,0.02,57355.0,-57317.0,117242.0,1408.00,78.57,28.0,M-SC,10.84,222.0,-1.00,0.85,1.98,OX40N,NTT,PAINTS
72,TATAELXSI,-2.59,-33.82,99.50,32.03,86812.0,-44587.0,87248.0,9161.00,-17.17,28.0,H-SC,9.27,158.0,-0.51,0.63,0.00,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,0.77,21.26,19.74,45.2,45896.0,40758.0,232500.0,1856.0,38.75,65.0,X-MC,7.41,79.0,0.89,1.68,29.5,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,0.55,5.34,11.71,17.67,18559.0,8030.0,158490.0,478.51,26.26,57.0,L-LC,11.56,185.0,0.43,1.15,27.23,XY25,ATH,MINING
26,FINCABLES,-1.11,3.35,101.17,107.91,154377.0,4948.0,152592.0,1641.55,-7.15,59.0,M-SC,7.92,220.0,0.03,1.11,14.80,OX40N,ATH,CABLES
50,MASFIN,-1.61,2.76,18.31,21.57,18435.0,2700.0,100680.0,397.04,-11.47,56.0,H-SC,6.16,167.0,0.15,0.73,46.92,XR,ATH,FINANCE
1,ABB,0.50,9.11,33.41,45.57,95371.0,23837.0,285456.0,7934.00,-29.92,65.0,H-MC,7.64,125.0,0.25,2.07,26.76,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,0.50,9.11,33.41,45.57,95371.0,23837.0,285456.0,7934.00,-29.92,65.0,H-MC,7.64,125.0,0.25,2.07,26.76,AR,NTT,ELECTRICAL
50,MASFIN,-1.61,2.76,18.31,21.57,18435.0,2700.0,100680.0,397.04,-11.47,56.0,H-SC,6.16,167.0,0.15,0.73,46.92,XR,ATH,FINANCE
26,FINCABLES,-1.11,3.35,101.17,107.91,154377.0,4948.0,152592.0,1641.55,-7.15,59.0,M-SC,7.92,220.0,0.03,1.11,14.80,OX40N,ATH,CABLES
39,INDIAMART,-0.92,-5.59,118.96,106.72,138518.0,-6895.0,116441.0,4810.62,-57.42,48.0,H-SC,7.97,140.0,-0.05,0.84,16.48,AR,ATH,MISC
85,ZYDUSLIFE,-0.22,-4.47,41.97,35.63,84030.0,-9359.0,200215.0,1286.17,-16.67,51.0,H-MC,5.09,120.0,-0.11,1.45,11.46,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-3.36,-12.67,53.71,34.24,146767.0,-39630.0,273258.0,1972.00,-31.42,18.0,X-LC,6.31,3.0,-0.27,1.98,0.00,X40,NTT,IT
33,HCLTECH,-4.14,-11.28,39.58,23.83,84953.0,-27289.0,214635.0,1908.19,-6.95,19.0,X-LC,3.81,8.0,-0.32,1.56,2.98,X40,ATH,IT
8,AWL,-1.58,-37.59,151.87,57.19,309110.0,-122591.0,203536.0,485.00,-64.34,20.0,X-SC,17.88,81.0,-0.40,1.47,0.00,XY24,NTT,FMCG
73,TCS,-3.18,-25.73,66.40,23.58,213341.0,-111333.0,321296.0,4311.59,-35.76,23.0,X-LC,6.96,2.0,-0.52,2.33,0.00,X40,ATH,IT
12,BATAINDIA,-1.13,-48.19,160.97,35.20,107304.0,-62009.0,66661.0,2096.00,-12.64,30.0,X-SC,21.87,91.0,-0.58,0.48,0.00,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,-0.06,4.02,16.10,20.77,39212.0,9408.0,243552.0,1644.00,-7.70,58.0,X-MC,0.56,64.0,0.24,1.76,11.72,X40N,NTT,BREWERIES
21,DABUR,0.20,0.77,42.61,43.70,106512.0,1901.0,249969.0,735.00,-8.57,53.0,X-MC,1.36,70.0,0.02,1.81,15.93,XY24,BTT,FMCG
36,ICICIGI,-1.15,2.31,16.65,19.34,34087.0,4619.0,204728.0,2252.93,-13.95,56.0,X-MC,1.68,66.0,0.14,1.48,18.61,X40,ATH,INSURANCE
65,SIEMENS,0.60,-10.74,40.62,25.51,67470.0,-19995.0,166100.0,4671.50,39.40,63.0,H-LC,2.46,51.0,-0.30,1.20,22.40,AR,ATH,ELECTRICAL
34,HINDUNILVR,-0.27,-9.01,24.93,13.67,58311.0,-23163.0,233900.0,2922.00,-32.42,47.0,X-LC,2.77,9.0,-0.40,1.69,10.42,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,-0.33,-25.50,108.69,55.47,196995.0,-62041.0,181245.0,181.5,-36.04,30.0,X-MC,17.72,60.0,-0.31,1.31,0.0,X40N,ATH,FINANCE
12,BATAINDIA,-1.13,-48.19,160.97,35.20,107304.0,-62009.0,66661.0,2096.0,-12.64,30.0,X-SC,21.87,91.0,-0.58,0.48,0.0,X40,NTT,FOOTWEAR
8,AWL,-1.58,-37.59,151.87,57.19,309110.0,-122591.0,203536.0,485.0,-64.34,20.0,X-SC,17.88,81.0,-0.40,1.47,0.0,XY24,NTT,FMCG
42,INFY,-3.36,-12.67,53.71,34.24,146767.0,-39630.0,273258.0,1972.0,-31.42,18.0,X-LC,6.31,3.0,-0.27,1.98,0.0,X40,NTT,IT
58,RELAXO,-0.64,-54.09,231.73,52.31,154439.0,-78514.0,66646.0,1176.0,-51.12,38.0,X-SC,14.00,92.0,-0.51,0.48,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-2.70,-33.26,114.03,42.84,49471.0,-21622.0,43384.0,424.00,-56.47,41.0,X-SC,17.91,80.0,-0.44,0.31,4.36,XY24,NTT,MISC
12,BATAINDIA,-1.13,-48.19,160.97,35.20,107304.0,-62009.0,66661.0,2096.00,-12.64,30.0,X-SC,21.87,91.0,-0.58,0.48,0.00,X40,NTT,FOOTWEAR
58,RELAXO,-0.64,-54.09,231.73,52.31,154439.0,-78514.0,66646.0,1176.00,-51.12,38.0,X-SC,14.00,92.0,-0.51,0.48,0.00,X40N,NTT,FOOTWEAR
51,MEDANTA,-1.90,-4.50,29.92,24.08,36276.0,-5707.0,121243.0,1486.00,-5.20,51.0,X-SC,9.37,87.0,-0.16,0.88,11.05,XY24,NTT,HEALTHCARE
35,HONAUT,-0.35,-22.74,85.50,43.32,107593.0,-37032.0,125840.0,58357.33,-32.13,43.0,X-SC,11.39,90.0,-0.34,0.91,2.18,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.74,-14.92,39.94,19.06,115590.0,-50758.0,289408.0,452.00,-53.50,47.0,X-LC,17.77,1.0,-0.44,2.10,4.79,X40,NTT,FMCG
73,TCS,-3.18,-25.73,66.40,23.58,213341.0,-111333.0,321296.0,4311.59,-35.76,23.0,X-LC,6.96,2.0,-0.52,2.33,0.00,X40,ATH,IT
42,INFY,-3.36,-12.67,53.71,34.24,146767.0,-39630.0,273258.0,1972.00,-31.42,18.0,X-LC,6.31,3.0,-0.27,1.98,0.00,X40,NTT,IT
75,TMPV,-0.18,-15.72,58.21,33.33,159831.0,-51223.0,274577.0,600.00,-82.53,59.0,X-LC,3.01,4.0,-0.32,1.99,12.25,XY24,NTT,AUTO
33,HCLTECH,-4.14,-11.28,39.58,23.83,84953.0,-27289.0,214635.0,1908.19,-6.95,19.0,X-LC,3.81,8.0,-0.32,1.56,2.98,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,0.10,-32.03,95.51,32.89,50121.0,-24730.0,52477.0,1800.00,-347.20,52.0,L-MC,11.45,258.0,-0.49,0.38,44.52,XR,NTT,BANKS
67,SONACOMS,-1.55,-8.70,52.35,39.09,48348.0,-8803.0,92356.0,804.02,-27.63,59.0,M-MC,9.56,193.0,-0.18,0.67,30.65,AR,ATH,AUTO
50,MASFIN,-1.61,2.76,18.31,21.57,18435.0,2700.0,100680.0,397.04,-11.47,56.0,H-SC,6.16,167.0,0.15,0.73,46.92,XR,ATH,FINANCE
18,COALINDIA,0.55,5.34,11.71,17.67,18559.0,8030.0,158490.0,478.51,26.26,57.0,L-LC,11.56,185.0,0.43,1.15,27.23,XY25,ATH,MINING
65,SIEMENS,0.60,-10.74,40.62,25.51,67470.0,-19995.0,166100.0,4671.50,39.40,63.0,H-LC,2.46,51.0,-0.30,1.20,22.40,AR,ATH,ELECTRICAL


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,-3.50,-51.32,199.32,45.70,171547.0,-90749.0,86066.0,26.4,-10.53,65.0,M-SC,18.82,204.0,-0.53,0.62,41.80,XY24,NTT,TRAVEL
1,ABB,0.50,9.11,33.41,45.57,95371.0,23837.0,285456.0,7934.0,-29.92,65.0,H-MC,7.64,125.0,0.25,2.07,26.76,AR,NTT,ELECTRICAL
7,ATULAUTO,0.14,-17.04,75.47,45.57,123424.0,-33592.0,163540.0,844.0,3543.24,55.0,M-SC,4.53,248.0,-0.27,1.18,23.76,XY24,NTT,AUTO
11,BANDHANBNK,3.02,-13.94,123.59,92.42,296277.0,-38833.0,239726.0,400.0,105.46,75.0,H-SC,6.52,174.0,-0.13,1.74,33.40,XY24,NTT,BANKS
83,WHIRLPOOL,-0.48,-26.23,146.62,81.92,124159.0,-30117.0,84681.0,2270.0,-56.86,60.0,M-SC,26.28,210.0,-0.24,0.61,18.95,XR,NTT,DURABLES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.62
1,25,45.51
2,50,77.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.06
MC    30.21
LC    24.67
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.06
X40      23.88
X40N     14.80
AR        9.02
XY25      8.83
XR        8.43
OX40N     7.08
SR        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.75
H-SC    22.15
X-LC    21.31
X-SC    11.49
M-SC    10.68
H-MC     5.02
M-MC     2.06
H-LC     1.20
L-LC     1.15
M-LC     1.01
L-SC     0.74
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.61,-14.53,53.27
FINANCE,13.09,-15.47,60.91
IT,11.20,-41.86,112.95
MISC,6.88,-36.50,92.33
ELECTRICAL,6.33,-7.43,46.63
PAINTS,5.06,-32.29,51.95
INSURANCE,4.84,-1.33,36.00
PHARMA,4.11,-3.51,36.09
AUTO,3.84,-17.65,62.51


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3391304.0,21
X40,1425223.0,15
AR,1326657.0,10
XR,1325441.0,12
X40N,1076606.0,10
OX40N,813575.0,10
XY25,397583.0,6
SR,303762.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3733907.0,24
M-SC,1442950.0,14
X-MC,1369696.0,15
X-LC,1333115.0,12
X-SC,1215620.0,10
H-MC,386475.0,3
L-SC,188293.0,2
M-LC,152101.0,1
M-MC,101844.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1258815.0      6
           AR         957151.0      5
           XR         852247.0      7
M-SC       XY24       834341.0      6
X-LC       X40        700903.0      6
X-SC       XY24       623860.0      4
X-MC       X40        542498.0      7
           X40N       434123.0      4
X-SC       X40N       409938.0      4
H-SC       OX40N      361932.0      4
M-SC       OX40N      355820.0      5
H-SC       SR         303762.0      2
X-LC       XY24       273057.0      2
           X40N       232545.0      2
H-MC       XY24       207074.0      1
X-MC       XY25       198918.0      2
           XY24       194157.0      2
X-SC       X40        181822.0      2
H-MC       AR         179401.0      2
M-SC       XR         178502.0      2
M-LC       XR         152101.0      1
X-LC       XY25       126610.0      2
L-SC       OX40N       95823.0      1
           XR          92470.0      1
M-SC       AR          74287.0      1
H-LC       AR          67470.0      1
M-MC       XY25        53496.0      1
L-MC       XR          50121.0      1
M-MC       AR          48348.0      1
L-LC       XY25        18559.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
